In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split,col, window

In [3]:
spark = SparkSession.builder.appName("Exemplo 1").getOrCreate()

In [5]:
socket_host= "servidor"
socket_port= 9999

In [6]:
lines_df = spark.readStream.format("socket").option("host",socket_host).option("port", socket_port).load() 

In [8]:
valores = split(col("value"),",")
dados_df = lines_df.withColumn("timestamp",valores[0]) \
        .withColumn("cidade", valores[1]) \
        .withColumn("temperatura", valores[2].cast("float"))

In [11]:
alertas_df = dados_df.filter(col("temperatura") > 20)

In [13]:
query = alertas_df.writeStream \
        .outputMode("append") \
        .format("console") \
        .start()
query.awaitTermination()    

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [12]:
query = dados_df.writeStream \
            .outputMode("append") \
            .format("console") \
            .start()
query.awaitTermination()    

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [24]:
valores = split(col("value"),",")
dados_df = lines_df.withColumn("timestamp_str",valores[0]) \
        .withColumn("cidade", valores[1]) \
        .withColumn("temperatura", valores[2].cast("float")) \
        .withColumn("timestamp",col("timestamp_str").cast("timestamp")) 

In [25]:
media_df =  dados_df \
            .withWatermark("timestamp","30 seconds") \
            .groupBy(
                window(col("timestamp"), "10 seconds"),
                col("cidade")
            ).avg("temperatura") 



In [26]:
query = media_df.writeStream \
            .outputMode("append") \
            .format("console") \
            .start()
query.awaitTermination() 

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 